In [ ]:
import torch
import transformers


tokenizer = transformers.AutoTokenizer.from_pretrained(
    "zhihan1996/DNABERT-2-117M", trust_remote_code=True
)
model = transformers.AutoModel.from_pretrained(
    "zhihan1996/DNABERT-2-117M", trust_remote_code=True
)

# input
dna_list = [                        # [B]
    "ACGTAGCATCGGATCTATCTATCGACACTTGGTTATCGATCTACGAGCATCTCGTTAGC", 
    #"CGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTA",
    #"TACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACG",
]
inputs = tokenizer(                 # [B, N]
    dna_list, return_tensors = 'pt', padding=True
)["input_ids"]
# output
hidden_states   = model(inputs)     # ( [B, N, 768], [B, 768] )
sequence_output = hidden_states[0]  # [B, N, 768]
pooled_output   = hidden_states[1]  # [B, 768]

# embedding with mean pooling
embedding_mean = torch.mean(sequence_output, dim=1)     # [B, 768]
# embedding with max  pooling
embedding_max = torch.max(sequence_output, dim=1)[0]    # [B, 768]